In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("/content/ORDER_PROD_CUS_REV.csv", encoding='utf-8')
df.head()

,order_id,order_item_id,order_purchase_timestamp,seller_id,p_product_id,product_category_name,price,freight_value,customer_id,customer_state,review_id,review_score
0,ORDER_75477,1,2018-09-13 08:59:02,SELLER_0452,PRODUCT_23128,cool_stuff,58.9,13.29,CUSTOMER_58023,RJ,REVIEW_45875,5
1,ORDER_63604,1,2018-04-26 10:53:06,SELLER_0414,PRODUCT_24351,pet_shop,239.9,19.93,CUSTOMER_30341,SP,REVIEW_24615,4
2,ORDER_05566,1,2019-01-14 14:33:31,SELLER_1637,PRODUCT_20226,furniture_decor,199.0,17.87,CUSTOMER_30949,MG,REVIEW_03775,5
3,ORDER_04632,1,2018-02-04 13:57:51,SELLER_1445,PRODUCT_07878,garden_tools,199.9,18.14,CUSTOMER_06713,SP,REVIEW_49155,5
4,ORDER_61452,1,2018-05-15 21:42:34,SELLER_0581,PRODUCT_03505,home_utilities,21.9,12.69,CUSTOMER_20396,MG,REVIEW_20134,4


In [3]:
df = df.drop(columns = ['order_item_id', 'p_product_id', 'customer_id', 'review_id'])
df.head()

,order_id,order_purchase_timestamp,seller_id,product_category_name,price,freight_value,customer_state,review_score
0,ORDER_75477,2018-09-13 08:59:02,SELLER_0452,cool_stuff,58.9,13.29,RJ,5
1,ORDER_63604,2018-04-26 10:53:06,SELLER_0414,pet_shop,239.9,19.93,SP,4
2,ORDER_05566,2019-01-14 14:33:31,SELLER_1637,furniture_decor,199.0,17.87,MG,5
3,ORDER_04632,2018-02-04 13:57:51,SELLER_1445,garden_tools,199.9,18.14,SP,5
4,ORDER_61452,2018-05-15 21:42:34,SELLER_0581,home_utilities,21.9,12.69,MG,4


In [4]:
# 데이터 전처리
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_year'] = df['order_purchase_timestamp'].dt.year
df['order_month'] = df['order_purchase_timestamp'].dt.month
df['order_day'] = df['order_purchase_timestamp'].dt.day
df['order_hour'] = df['order_purchase_timestamp'].dt.hour


In [5]:
# 계절 정보를 컬럼으로 추가
# 봄 : 9, 10, 11, 여름 : 12, 1, 2, 가을 : 3, 4, 5, 겨울 : 6, 7, 8
def get_season(month):
    if month in [12, 1, 2]:
        return 'Summer'
    elif month in [3, 4, 5]:
        return 'Autumn'
    elif month in [6, 7, 8]:
        return 'Winter'
    else:
        return 'Spring'

df['season'] = df['order_month'].apply(get_season)
df.head()

,order_id,order_purchase_timestamp,seller_id,product_category_name,price,freight_value,customer_state,review_score,order_year,order_month,order_day,order_hour,season
0,ORDER_75477,2018-09-13 08:59:02,SELLER_0452,cool_stuff,58.9,13.29,RJ,5,2018,9,13,8,Spring
1,ORDER_63604,2018-04-26 10:53:06,SELLER_0414,pet_shop,239.9,19.93,SP,4,2018,4,26,10,Autumn
2,ORDER_05566,2019-01-14 14:33:31,SELLER_1637,furniture_decor,199.0,17.87,MG,5,2019,1,14,14,Summer
3,ORDER_04632,2018-02-04 13:57:51,SELLER_1445,garden_tools,199.9,18.14,SP,5,2018,2,4,13,Summer
4,ORDER_61452,2018-05-15 21:42:34,SELLER_0581,home_utilities,21.9,12.69,MG,4,2018,5,15,21,Autumn


In [6]:
df['order_date'] = df['order_purchase_timestamp'].dt.date
df['order_day_of_week'] = df['order_purchase_timestamp'].dt.dayofweek
df.head()

,order_id,order_purchase_timestamp,seller_id,product_category_name,price,freight_value,customer_state,review_score,order_year,order_month,order_day,order_hour,season,order_date,order_day_of_week
0,ORDER_75477,2018-09-13 08:59:02,SELLER_0452,cool_stuff,58.9,13.29,RJ,5,2018,9,13,8,Spring,2018-09-13,3
1,ORDER_63604,2018-04-26 10:53:06,SELLER_0414,pet_shop,239.9,19.93,SP,4,2018,4,26,10,Autumn,2018-04-26,3
2,ORDER_05566,2019-01-14 14:33:31,SELLER_1637,furniture_decor,199.0,17.87,MG,5,2019,1,14,14,Summer,2019-01-14,0
3,ORDER_04632,2018-02-04 13:57:51,SELLER_1445,garden_tools,199.9,18.14,SP,5,2018,2,4,13,Summer,2018-02-04,6
4,ORDER_61452,2018-05-15 21:42:34,SELLER_0581,home_utilities,21.9,12.69,MG,4,2018,5,15,21,Autumn,2018-05-15,1


In [7]:
# 0: 주중, 1: 주말
df['is_weekend'] = df['order_purchase_timestamp'].dt.dayofweek.apply(lambda x: 1 if x >= 5 else 0)
df.head()

,order_id,order_purchase_timestamp,seller_id,product_category_name,price,freight_value,customer_state,review_score,order_year,order_month,order_day,order_hour,season,order_date,order_day_of_week,is_weekend
0,ORDER_75477,2018-09-13 08:59:02,SELLER_0452,cool_stuff,58.9,13.29,RJ,5,2018,9,13,8,Spring,2018-09-13,3,0
1,ORDER_63604,2018-04-26 10:53:06,SELLER_0414,pet_shop,239.9,19.93,SP,4,2018,4,26,10,Autumn,2018-04-26,3,0
2,ORDER_05566,2019-01-14 14:33:31,SELLER_1637,furniture_decor,199.0,17.87,MG,5,2019,1,14,14,Summer,2019-01-14,0,0
3,ORDER_04632,2018-02-04 13:57:51,SELLER_1445,garden_tools,199.9,18.14,SP,5,2018,2,4,13,Summer,2018-02-04,6,1
4,ORDER_61452,2018-05-15 21:42:34,SELLER_0581,home_utilities,21.9,12.69,MG,4,2018,5,15,21,Autumn,2018-05-15,1,0


In [8]:
practice_selected = ['order_date', 'product_category_name', 'season', 'is_weekend']
practice_df = df[practice_selected]
practice_df.head()

,order_date,product_category_name,season,is_weekend
0,2018-09-13,cool_stuff,Spring,0
1,2018-04-26,pet_shop,Autumn,0
2,2019-01-14,furniture_decor,Summer,0
3,2018-02-04,garden_tools,Summer,1
4,2018-05-15,home_utilities,Autumn,0


In [9]:
practice_df['order_date'] = pd.to_datetime(practice_df['order_date'])

demand_df = practice_df.groupby(['order_date', 'product_category_name', 'season', 'is_weekend']).size().reset_index(name='demand')

demand_df.head(10)

,order_date,product_category_name,season,is_weekend,demand
0,2017-10-03,fashion_shoes,Spring,0,1
1,2017-10-03,furniture_decor,Spring,0,2
2,2017-10-03,sports_leisure,Spring,0,3
3,2017-10-03,toys,Spring,0,1
4,2017-10-03,watches_gifts,Spring,0,1
5,2017-10-04,Unknown,Spring,0,1
6,2017-10-04,air_conditioning,Spring,0,4
7,2017-10-04,automotive,Spring,0,1
8,2017-10-04,baby,Spring,0,3
9,2017-10-04,bed_bath_table,Spring,0,2


In [10]:
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

test_start_date = '2019-07-01'
test_end_date = '2019-07-31'

test_data = demand_df[(demand_df['order_date'] >= test_start_date) & (demand_df['order_date'] <= test_end_date)]

# 2019년 7월 데이터 제거
train_end_date = '2019-06-30'
demand_df_filtered = demand_df[demand_df['order_date'] <= train_end_date]

def map_season(season):
    if season == 'Spring':
        return 1
    elif season == 'Summer':
        return 2
    elif season == 'Autumn':
        return 3
    elif season == 'Winter':
        return 4
    else:
        return 0  # 예외 처리

demand_df_filtered['season'] = demand_df_filtered['season'].apply(map_season)

def is_weekend(date):
    day_of_week = date.weekday()  # 0: 월요일, 6: 일요일
    return 1 if day_of_week >= 5 else 0  # 토요일(5), 일요일(6)은 주말로 처리

category_groups = demand_df_filtered.groupby('product_category_name')

results = pd.DataFrame()

In [11]:
scalers = {
    'y': MinMaxScaler(feature_range=(0, 1)),
    'season': MinMaxScaler(feature_range=(0, 1)),
    'is_weekend': MinMaxScaler(feature_range=(0, 1))
}

In [12]:
# 타임 스텝 설정 (최근 날짜로부터 기간 정함)
time_step = 450

class DemandDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

for category, group in tqdm(category_groups, desc="Processing categories"):
    if category in test_data['product_category_name'].unique():  # 테스트 데이터셋에 해당 카테고리가 있는 경우에만 예측 수행

        group = group.rename(columns={'order_date': 'ds', 'demand': 'y', 'season': 'season'})
        group['is_weekend'] = group['ds'].apply(is_weekend)

        features = group[['y', 'season', 'is_weekend']]
        target = group['y']

        features_scaled = features.copy()
        for feature in features.columns:
            features_scaled[feature] = scalers[feature].fit_transform(features[[feature]])

        target_scaled = scalers['y'].fit_transform(target.values.reshape(-1, 1))

        X = []
        y = []
        for i in range(len(features_scaled) - time_step):
            X.append(features_scaled.iloc[i:i + time_step].values)
            y.append(target_scaled[i + time_step])

        X = np.array(X)
        y = np.array(y)

        if X.shape[0] == 0 or X.shape[1] == 0:
            print(f"Skipping category {category} due to insufficient data.")
            continue

        dataset = DemandDataset(X, y)
        dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

        class LSTMModel(nn.Module):
            def __init__(self, input_size, hidden_size, num_layers):
                super(LSTMModel, self).__init__()
                self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
                self.fc = nn.Linear(hidden_size, 1)

            def forward(self, x):
                h_0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)
                c_0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)
                out, _ = self.lstm(x, (h_0, c_0))
                out = self.fc(out[:, -1, :])
                return out

        input_size = X.shape[2]
        hidden_size = 50
        num_layers = 2
        num_epochs = 20
        learning_rate = 0.001

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = LSTMModel(input_size, hidden_size, num_layers).to(device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        model.train()
        for epoch in range(num_epochs):
            for inputs, labels in dataloader:
                inputs = inputs.to(device).float()
                labels = labels.to(device).float()

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            if (epoch+1) % 10 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

        future_dates = pd.date_range(start='2019-07-01', end='2019-07-31', freq='D')
        future = pd.DataFrame({'ds': future_dates})

        future['season'] = 4
        future['is_weekend'] = future['ds'].apply(is_weekend)

        future_features = future[['season', 'is_weekend']]
        future_features['y'] = 0  # 미래 데이터에 y를 0으로 설정
        for feature in future_features.columns:
            future_features[feature] = scalers[feature].transform(future_features[[feature]])

        # 마지막 훈련 데이터로부터 미래 예측을 위한 입력 데이터 생성
        future_X = []
        last_train_data = features_scaled.iloc[-time_step:].values
        for i in range(len(future_features)):
            input_data = np.vstack([last_train_data, future_features.iloc[:i+1].values])[-time_step:]
            future_X.append(input_data)

        future_X = np.array(future_X)

        model.eval()
        with torch.no_grad():
            future_X_tensor = torch.tensor(future_X).to(device).float()
            future_predictions = model(future_X_tensor)
            future_predictions = future_predictions.cpu().numpy()
            future_predictions = scalers['y'].inverse_transform(future_predictions)

        future['product_category_name'] = category
        future['order_date'] = future['ds']
        future['demand'] = future_predictions
        results = pd.concat([results, future[['order_date', 'product_category_name', 'demand']]])

Processing categories:   0%|          | 0/72 [00:00<?, ?it/s]

Skipping category Unknown due to insufficient data.
Skipping category agro_industry_and_commerce due to insufficient data.
Skipping category air_conditioning due to insufficient data.
Skipping category arts due to insufficient data.
Skipping category arts_and_crafts due to insufficient data.
Skipping category audio due to insufficient data.


Processing categories:  10%|▉         | 7/72 [00:03<00:31,  2.05it/s]

Epoch [10/20], Loss: 0.0187
Epoch [20/20], Loss: 0.0053


Processing categories:  11%|█         | 8/72 [00:03<00:27,  2.29it/s]

...

Epoch [10/20], Loss: 0.0058
Epoch [20/20], Loss: 0.0056
Epoch [10/20], Loss: 0.0127


Processing categories:  94%|█████████▍| 68/72 [00:07<00:00, 18.43it/s]

Epoch [20/20], Loss: 0.0110
Skipping category technical_books due to insufficient data.
Epoch [10/20], Loss: 0.0027
Epoch [20/20], Loss: 0.0014
Epoch [10/20], Loss: 0.0017


Processing categories: 100%|██████████| 72/72 [00:07<00:00,  9.31it/s]

Epoch [20/20], Loss: 0.0015
Epoch [10/20], Loss: 0.0218
Epoch [20/20], Loss: 0.0232


In [13]:
len(results)

558

In [14]:
results.tail()

,order_date,product_category_name,demand
26,2019-07-27,watches_gifts,19.521379
27,2019-07-28,watches_gifts,19.763075
28,2019-07-29,watches_gifts,19.747938
29,2019-07-30,watches_gifts,19.640581
30,2019-07-31,watches_gifts,19.517929


In [15]:
# 모든 날짜와 카테고리의 조합 생성
all_dates = pd.date_range(start=test_start_date, end=test_end_date, freq='D')
all_categories = test_data['product_category_name'].unique()
all_combinations = pd.MultiIndex.from_product([all_dates, all_categories], names=['order_date', 'product_category_name']).to_frame(index=False)

results = pd.merge(all_combinations, results, on=['order_date', 'product_category_name'], how='left')
test_data = pd.merge(all_combinations, test_data, on=['order_date', 'product_category_name'], how='left')


merged_results = pd.merge(results, test_data, on=['order_date', 'product_category_name'], how='inner')

In [16]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score, mean_squared_error

mae = mean_absolute_error(merged_results['demand_x'].fillna(0), merged_results['demand_y'].fillna(0))
r2 = r2_score(merged_results['demand_x'].fillna(0), merged_results['demand_y'].fillna(0))
rmse = mean_squared_error(merged_results['demand_x'].fillna(0), merged_results['demand_y'].fillna(0), squared=False)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R2) Score: {r2}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

Mean Absolute Error (MAE): 2.1394143407457222
R-squared (R2) Score: 0.533101365323076
Root Mean Squared Error (RMSE): 4.559684581913827


In [17]:
# 전체 테스트 데이터에 대한 기술 통계량 계산
test_data_stats = test_data['demand'].describe()
predicted_stats = results['demand'].describe()

print("\nActual Demand Statistics (Overall):")
print(test_data_stats)

print("\nPredicted Demand Statistics (Overall):")
print(predicted_stats)


Actual Demand Statistics (Overall):
count    920.000000
mean       5.055435
std        5.919865
min        1.000000
25%        1.000000
50%        3.000000
75%        7.000000
max       40.000000
Name: demand, dtype: float64

Predicted Demand Statistics (Overall):
count    558.000000
mean      12.251292
std        7.236359
min        2.722226
25%        6.549270
50%        9.682943
75%       18.971352
max       26.136734
Name: demand, dtype: float64


In [18]:
# 결측치가 많은 것 같다
print("결측치 수 :")
print(results.isnull().sum())

결측치 수 :
order_date                  0
product_category_name       0
demand                   1457
dtype: int64


In [19]:
# 확인할 결측치 위치
missing_demand = results[results['demand'].isnull()]
print("결측치 행:")
print(missing_demand)

결측치 행:
     order_date            product_category_name  demand
0    2019-07-01                 air_conditioning     NaN
1    2019-07-01                             arts     NaN
6    2019-07-01                   consoles_games     NaN
7    2019-07-01  construction_tools_construction     NaN
8    2019-07-01      construction_tools_lighting     NaN
...         ...                              ...     ...
2010 2019-07-31                  diapers_hygiene     NaN
2011 2019-07-31                  arts_and_crafts     NaN
2012 2019-07-31                          flowers     NaN
2013 2019-07-31            fashion_mens_clothing     NaN
2014 2019-07-31                bedroom_furniture     NaN

[1457 rows x 3 columns]
